#  **UserkNN  CV для сравнения моделей**

# Подготовка

In [2]:
!pip install rectools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 15.4 MB/s eta 0:00:00


In [23]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
from pprint import pprint

import numpy as np
import pandas as pd
from implicit.nearest_neighbours import (
    BM25Recommender,
    CosineRecommender,
    TFIDFRecommender,
)
from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import (
    MAP,
    NDCG,
    MeanInvUserFreq,
    Precision,
    Recall,
    Serendipity,
    calc_metrics,
)
from rectools.model_selection import TimeRangeSplitter
from tqdm.auto import tqdm

from experiments.models.userknn import UserKnn

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 200)

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## Кросс-валидация

In [5]:
def cros_val(interactions, cv, models, metrics):
    results = []

    fold_iterator = cv.split(interactions, collect_fold_stats=True)

    for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
        print(f"\n==================== Fold {i_fold}")
        pprint(fold_info)

        df_train = interactions.df.iloc[train_ids].copy()
        df_test = interactions.df.iloc[test_ids][Columns.UserItem].copy()

        catalog = df_train[Columns.Item].unique()

        for model_name, model in models.items():
            userknn_model = UserKnn(model=model, N_users=50, popular_model=PopularModel())
            userknn_model.fit(df_train)

            recos = userknn_model.predict(df_test)

            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )

            fold = {"fold": i_fold, "model": model_name}
            fold.update(metric_values)
            results.append(fold)
    return pd.DataFrame(results).groupby("model").mean()

# Эксперименты

## Загружаем датасет KION

In [25]:
# DATA_PATH = "../../kion_train/interactions.csv"
DATA_PATH = "/content/drive/MyDrive/kion_train/interactions.csv"


interactions_df = pd.read_csv(DATA_PATH, parse_dates=["last_watch_dt"])

interactions_df.rename(
    columns={"last_watch_dt": Columns.Datetime, "total_dur": Columns.Weight},
    inplace=True,
)

interactions = Interactions(interactions_df)
# interactions = Interactions(interactions_df.sample(frac=0.01))
interactions_df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


## Задаем фолды для кросс-валидации

In [26]:
print(f"Min date: {interactions_df.datetime.min()}")
print(f"Max date: {interactions_df.datetime.max()}")

Min date: 2021-03-13 00:00:00
Max date: 2021-08-22 00:00:00


In [27]:
# Уменьшим число фолдов, т.к. модель будет долго обучаться. Кажется, что 5 будет достаточно.
N_SPLITS = 5
# Тестовую часть сделаем размером с две недели.
# Больше взять вряд ли получится, т.к. на последних фолдах на трейн останется слишком мало.
TEST_SIZE = "14D"

In [28]:
# Init generator of folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=False,  # теперь мы умеем работать с холодными пользователями
)

In [29]:
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-06-14 00:00:00', freq='14D'),
  Timestamp('2021-06-28 00:00:00', freq='14D')),
 (Timestamp('2021-06-28 00:00:00', freq='14D'),
  Timestamp('2021-07-12 00:00:00', freq='14D')),
 (Timestamp('2021-07-12 00:00:00', freq='14D'),
  Timestamp('2021-07-26 00:00:00', freq='14D')),
 (Timestamp('2021-07-26 00:00:00', freq='14D'),
  Timestamp('2021-08-09 00:00:00', freq='14D')),
 (Timestamp('2021-08-09 00:00:00', freq='14D'),
  Timestamp('2021-08-23 00:00:00', freq='14D'))]

## Задаем метрики и модели, по которым будем делать CV

In [30]:
# Добавим побольше метрик
metrics = {
    "precision@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "MAP@10": MAP(k=10),
    "NDCG@10": NDCG(k=10),
    "novelty@10": MeanInvUserFreq(k=10),
    "serendipity@10": Serendipity(k=10),
}

# Добавим BM25Recommender
models = {
    "cosine_userknn": CosineRecommender(),  # implicit
    "tfidf_userknn": TFIDFRecommender(),
    "bm25_userknn": BM25Recommender(),
}

## CV

In [31]:
%%time
results = cros_val(interactions, cv, models, metrics)


==================== Fold 0
{'end': Timestamp('2021-06-28 00:00:00', freq='14D'),
 'i_split': 0,
 'start': Timestamp('2021-06-14 00:00:00', freq='14D'),
 'test': 573256,
 'test_items': 7169,
 'test_users': 212510,
 'train': 1979424,
 'train_items': 13649,
 'train_users': 439529}


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.15247178077697754 seconds
  warnings.warn(


  0%|          | 0/439529 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.188795804977417 seconds
  warnings.warn(


  0%|          | 0/439529 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.42634081840515137 seconds
  warnings.warn(


  0%|          | 0/850489 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.38304781913757324 seconds
  warnings.warn(


  0%|          | 0/850489 [00:00<?, ?it/s]

CPU times: user 7h 47min 57s, sys: 3min 49s, total: 7h 51min 46s
Wall time: 5h 31min 3s


In [ ]:
## Анализ результатов

In [32]:
results

,fold,precision@10,recall@10,NDCG@10,MAP@10,novelty@10,serendipity@10
model,,,,,,,
bm25_userknn,2.0,0.046358,0.261871,0.035866,0.046052,6.242064,0.000052
cosine_userknn,2.0,0.048716,0.272779,0.037556,0.047615,5.287692,0.000032
tfidf_userknn,2.0,0.050042,0.279642,0.038691,0.049217,5.391348,0.000034


По всем метрикам, кроме novelty и serendipity лучше оказалась `tfidf_userknn`. Можно сказать, что эта модель дает более типичные и "скучные", но точные и "проверенные" рекомендации. Для `bm25_userknn` ситуация обратная. Какую модель выбрать в итоге зависит от задачи. У нас задача выбить MAP@10 на лидерборде😅, поэтому логичнее брать TF-IDF.

# Лучшую модель обучаем на всем датасете и сохраняем

In [34]:
userknn_model = UserKnn(model=TFIDFRecommender(), N_users=50, popular_model=PopularModel())
userknn_model.fit(interactions.df)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.5924592018127441 seconds
  warnings.warn(


  0%|          | 0/962179 [00:00<?, ?it/s]

In [33]:
# MODEL_PATH = '../../models/user_knn.pkl'
MODEL_PATH = "/content/drive/MyDrive/recsys/user_knn.pkl"

In [35]:
import pickle

pickle.dump(userknn_model, open(MODEL_PATH, "wb"))

In [ ]:
# Проверим, что все работает
pickled_model = pickle.load(open(MODEL_PATH, "rb"))
pickled_model.recommend(176549)

[11345, 6809, 142, 4880, 2657, 3734, 4151, 13865, 9728, 10440]